In [7]:
# 在 Colab 安裝必要套件
!pip install openai-whisper gradio gtts numpy
!pip install --upgrade "gradio>=3.50"
# !pip install pyngrok

# !pip install "gradio==4.44.1" "huggingface_hub<0.24"
# !pip install "gradio==4.44.1" "huggingface_hub==0.23.4" "fastapi==0.110.0" "uvicorn==0.29.0"

In [8]:
# 載入 Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
# 建立資料夾（如果不存在）
os.makedirs("/content/drive/MyDrive/2025 AI 辯論", exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import whisper, gradio as gr, os, numpy as np, tempfile, wave, soundfile as sf
from gtts import gTTS

# 載入模型
print("正在載入 Whisper small 模型...")
model = whisper.load_model("small")
print("模型載入完成！")

# 固定預覽檔
preview_path = "/content/preview_recording.mp3"  # 直接用 mp3 預覽！

# AI 回應
def mock_ai_response(text):
    return f"我反駁你：'{text}' 這個觀點完全錯誤！讓我來拆穿你..."

# 處理上傳的 mp3
def process(filepath):
    if not filepath:
        return "請上傳 mp3 音檔", gr.Audio(visible=False), None

    try:
        # 直接讀 mp3
        data, sr = sf.read(filepath)
        if len(data.shape) > 1:
            data = np.mean(data, axis=1) # 轉單聲道
        data = data.astype(np.float32)

        energy = np.mean(np.abs(data))
        if len(data) < sr * 0.5:
            return "錄音太短（少於0.5秒）", gr.Audio(visible=False), None
        if energy < 0.001:
            return f"音量太小（能量 {energy:.5f}）", gr.Audio(visible=False), None

        sf.write(preview_path, data, sr)
        return f"載入成功！能量: {energy:.4f}", preview_path, {"data": data, "sample_rate": sr}

    except Exception as e:
        return f"讀檔失敗：{e}", gr.Audio(visible=False), None

def debate(state):
    if not state:
        return "請先載入音檔", "", None

    data, sr = state["data"], state["sample_rate"]

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        with wave.open(f.name, 'w') as wav:
            wav.setnchannels(1)
            wav.setsampwidth(2)
            wav.setframerate(sr)
            wav.writeframes((data * 32767).astype(np.int16).tobytes())
        temp_path = f.name

    try:
        result = model.transcribe(temp_path, language="zh")
        text = result["text"].strip()
    except Exception as e:
        return f"語音辨識失敗：{e}", "", None
    finally:
        if os.path.exists(temp_path):
            os.unlink(temp_path)

    if not text:
        return "我聽不懂你說了什麼", "", None

    response = mock_ai_response(text)
    tts = gTTS(response, lang='zh-tw')
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as f:
        tts.save(f.name)
        mp3_path = f.name

    return text, response, mp3_path

def save_to_drive(state):
    if not state:
        return "沒有音檔可儲存"
    path = "/content/drive/MyDrive/2025 AI 辯論/debate_sample.wav"
    sf.write(path, state["data"], state["sample_rate"])
    return f"已儲存到 Drive！"

# === Gradio 介面 ===
with gr.Blocks(title="AI 語音辯論上傳 mp3 版本") as demo:
    gr.Markdown("# AI 語音辯論系統（Colab 專用）")
    gr.Markdown("**支援任何格式上傳（mp3/m4a/wav/amr）＋ 預覽永不消失！**")

    state = gr.State()

    with gr.Row():
        audio_input = gr.Audio(
            label="上傳任何錄音檔（mp3）",
            type="filepath",
            format="mp3"
        )
        load_btn = gr.Button("載入音檔", variant="secondary")

    status = gr.Textbox(label="狀態", interactive=False)
    preview = gr.Audio(label="你的錄音預覽（點播放，永不消失！）", type="filepath", interactive=True)

    with gr.Row():
        debate_btn = gr.Button("送出辯論", variant="primary")
        save_btn = gr.Button("儲存到 Drive")

    with gr.Row():
        user_text = gr.Textbox(label="你說了什麼", interactive=False)
        ai_text = gr.Textbox(label="AI 反駁", interactive=False)
        ai_audio = gr.Audio(label="聽 AI 說話", type="filepath")

    # 事件綁定
    load_btn.click(process, inputs=audio_input, outputs=[status, preview, state])
    debate_btn.click(debate, inputs=state, outputs=[user_text, ai_text, ai_audio])
    save_btn.click(save_to_drive, inputs=state, outputs=status)

# 啟動
demo.launch(share=True, debug=True)

正在載入 Whisper small 模型...
模型載入完成！
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c6329cbb7389725f0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c6329cbb7389725f0f.gradio.live
